<a href="https://colab.research.google.com/github/srhr17/MachineHack/blob/master/Starter_Notebook_For_Participants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Required Files

In [0]:
import pandas as pd
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

data_folder = ''

In [0]:
all_files = os.listdir(data_folder)
print('Total Number of Files :', len(all_files))

Total Number of Files : 2827


# Read Train Files

In [0]:
train_df = pd.read_csv('Train.csv')
train_df

,File_Name,Labels
0,file_2180.txt,8
1,file_693.txt,4
2,file_2469.txt,6
3,file_2542.txt,6
4,file_378.txt,16
...,...,...
1973,file_1930.txt,19
1974,file_1821.txt,19
1975,file_350.txt,16
1976,file_1933.txt,19


In [0]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
10      2
18      1
Name: Labels, dtype: int64

In [0]:
test_df = pd.read_csv('/Users/anurag/Documents/Workspace/Our_test.csv')
test_df

,File_Name,Labels
0,file_2300.txt,6
1,file_809.txt,4
2,file_1383.txt,0
3,file_983.txt,1
4,file_1713.txt,19
...,...,...
844,file_2474.txt,6
845,file_863.txt,4
846,file_1547.txt,20
847,file_1292.txt,0


In [0]:
test_df.Labels.value_counts()

6     174
19    112
4     104
0      87
5      60
15     58
1      50
16     46
11     45
8      34
14     32
7      12
2      10
20      8
13      7
21      4
9       1
12      1
3       1
17      1
18      1
10      1
Name: Labels, dtype: int64

In [0]:
print(train_df.Labels.nunique(), test_df.Labels.nunique())

22 22


In [0]:
## let's read the text scripts in the train and test dataframes..

train_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in train_df['File_Name']]
test_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in test_df['File_Name']]

# Lets look at a script file after Reading..

In [0]:
#lets check one of the scripts..
print(train_df['Script'][4][:3000])

<b><!--

</b>if (window!= top)

top.location.href=location.href

<b>// -->

</b>

The Abyss - by James Cameron 

                                  THE ABYSS

                            AN ORIGINAL SCREENPLAY

                                      BY

                                JAMES CAMERON

                               August 2, 1988

                             Director's Revision

------------------------------------------------------------------------------

                                  THE ABYSS

OMITTED                                                                 1

OMITTED                                                                 2

TITLE: THE ABYSS -- ON BLACK, DISSOLVING TO COBALT BLUE

EXT. OCEAN/UNDERWATER -- DAY                                            3

Blue, deep and featureless, the twilight of five hundred feet down.

PROPELLER SOUND.  Materializing out of the blue limbo is the enormous but

sleek form of an Ohio-class SSBN ballistic missile su

In [0]:
print(test_df['Script'][4][:3000])

                                          KING KONG

                                          Written by

                        Fran Walsh, Philippa Boyens and Peter Jackson

                                     Based on a Story by

                             Merian C. Cooper and Edgar Wallace

                                                                   1.

          EXT. CENTRAL PARK - DAY

          CLOSE ON: A scrawny MONKEY scratches.

          ANGLES ON: Defeated, listless ANIMALS, in the bleak environs of a

          dilapidated ZOO.

          WIDER: It is CENTRAL PARK ZOO in depression era NEW YORK. The PARK

          itself is like a GARBAGE DUMP, dotted with squalid SHANTY TOWNS.

          Against these BLEAK IMAGES, the SOUND of a BRIGHT, BRASSY SONG

          fades up: Al Jolson, singing "I'm Sitting on Top of the World".

          The sky line of MANHATTAN rises in the background, a grim steaming

          jungle on this cold FALL day.

                 

In [0]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
10      2
18      1
Name: Labels, dtype: int64

In [0]:
# !pip install keras
# !pip install nltk

### There is single instance for one class (Label == 18), lets duplicate the row for stratification

In [0]:
train_df = train_df.append(train_df[train_df['Labels'] == 18])
train_df.reset_index(drop=True, inplace=True)

In [0]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
18      2
10      2
Name: Labels, dtype: int64

# Import the Modeling Libraries 

In [0]:
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords


Using TensorFlow backend.
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/anurag/opt/anaconda3/lib/python3.7

Let's load the datasets

In [0]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anurag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Define the Scoring Metric

In [0]:
from sklearn.metrics import log_loss

### Let's Use LabelEncoder from scikit-learn to convert Genre labels to integers, 0, 1 2

In [0]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train_df.Labels.values)

### Before going further it is important that we split the data into training and validation sets. We can do it using 
#### `train_test_split` from the `model_selection` module of scikit-learn.

In [0]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.Script.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.30, shuffle=True)

In [0]:
print (xtrain.shape)
print (xvalid.shape)

(1385,)
(594,)


# Building Basic Models

### Let's start building our very first model. 

### Our very first model is a simple TF-IDF (Term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression.

In [0]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

## TFIDF on test data

In [0]:
x_test_tfv = tfv.transform(test_df['Script'].values)

In [0]:
## Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print("logloss: %0.3f " % log_loss(yvalid, predictions))

# Submission 

### Let's predict on the entire test data...

In [0]:
test_set_preds = pd.DataFrame(columns = train_df.Labels.unique().tolist())
test_set_preds.insert(0, 'File_Name', test_df.File_Name)

In [0]:
test_set_preds[test_df.Labels.unique().tolist()] = clf.predict_proba(x_test_tfv)

In [0]:
test_set_preds.shape

(849, 23)

In [0]:
test_set_preds.head()

,File_name,8,4,6,16,15,19,1,14,0,...,7,3,13,20,21,12,10,17,9,18
0,file_2300.txt,0.00476286,0.0421466,0.0621472,0.00388802,0.0353658,0.127008,0.0038829,0.0455528,0.0130443,...,0.0364149,0.0101705,0.0726348,0.014007,0.00432342,0.042123,0.00382821,0.00659258,0.00388578,0.201259
1,file_809.txt,0.00473483,0.0424302,0.0618728,0.00380366,0.0337543,0.162379,0.00384745,0.0406282,0.0128616,...,0.0326453,0.0099209,0.155985,0.0140639,0.00426309,0.0393143,0.00377858,0.00636009,0.00379855,0.0948981
2,file_1383.txt,0.00493601,0.144602,0.0975445,0.00401702,0.0355864,0.0758506,0.00399558,0.0460768,0.0133359,...,0.0342179,0.0110146,0.0529838,0.0142812,0.00442489,0.151729,0.00395552,0.00689023,0.00399736,0.0972641
3,file_983.txt,0.00499925,0.049697,0.081371,0.00403428,0.0363712,0.103302,0.00402333,0.0467117,0.0135967,...,0.036615,0.0109523,0.0638244,0.0146887,0.00453059,0.0479881,0.00401123,0.00689013,0.00404956,0.225464
4,file_1713.txt,0.00506564,0.110646,0.153704,0.00412486,0.0376808,0.0723873,0.00410832,0.0462569,0.0141907,...,0.0720046,0.0111348,0.0489596,0.0153917,0.00459447,0.0497823,0.00411517,0.00707706,0.00413493,0.0888852


In [0]:
#Reorder the columns to match the Sample_submission_file
test_set_preds = test_set_preds[['File_name',0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

In [0]:
#Write your submissions to an excel file
test_set_preds.to_excel('test_set_preds_v1.0.xlsx', index=False)

#### Follow the sample submission format for your submission file

#### Please verify the following before submitting your solution to avoid an invalid sibmission.

1. The format of the file is excel(.xlsx)

2. The file doesn’t contain additional styling elements such as bold headings or table borders

3. The length of the submission exactly matches with that of the sample submission and test set

4. The file name doesnot have any spaces or special characters

5. All the columns are present